In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from tqdm.notebook import tqdm

### Data Scraping

In [2]:
HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36', 'accept': '*/*'}
COOKIES = { 'birthtime': '283993201', 'mature_content': '1' }

In [4]:
start = 0
pace = 50
count = 54700

In [5]:
# собираем все ссылки на игры в стиме
links = []
while start + pace < count:
    batch_url = f'https://store.steampowered.com/search/results/?query&start={start}&count=50&dynamic_data=&category1=998&snr=1_7_7_230_7&infinite=1'
    batch_page = rq.get(batch_url)
    print(batch_page)
    batch_soup = BeautifulSoup(batch_page.text, 'html.parser')
    for app in batch_soup.find_all('a'):
        links.append(app['href'])
    if start % 1000 == 0:
        print(f'Done {start + pace} out of {count}')
    start += pace

<Response [200]>
Done 50 out of 54700
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Done 1050 out of 54700
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Done 2050 out of 54700
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]

In [7]:
# приводим ссылки к рабочему виду
pat = re.compile('(?<=").+(?=")')
links = [pat.search(app).group() for app in links]

pat2 = re.compile(r'\\')
links = [re.sub(pat2, '', app) for app in links]

In [20]:
# содержимое html страниц
def get_content(url):
    game_page = rq.get(url, headers=HEADERS, cookies=COOKIES)
    if game_page.status_code != 200:
        print(game_page, url)
    game_soup = BeautifulSoup(game_page.text, 'html.parser')
    return game_soup

In [21]:
# необходимая информация по играм
def get_info(soup_page):
    desc = ''
    reviews = []
    tags = ''
    content = ''
    details = ''
    full_desc = ''
    try:
        desc = soup_page.find('div', {'class': 'game_description_snippet'}).text # Game description
    except AttributeError:
        pass
    reviews.extend(soup_page.find_all('div', {'class': 'user_reviews_summary_row'})) # review - 1st - recent, 2nd - all, recent reviews can be absent
    try:
        tags = soup_page.find('div', {'class': 'glance_tags popular_tags'}).text #tags
    except AttributeError:
        pass
    try:
        content = soup_page.find('div', {'id': 'game_area_content_descriptors'}).text #content description
    except AttributeError:
        pass
    try:
        details = soup_page.find(lambda tag: tag.name == 'div' and tag.get('class') == ['details_block']).text #Title, Genre, Developer, Publisher, Release Date
    except AttributeError:
        pass
    try:
        full_desc = soup_page.find('div', {'id': 'game_area_description'}).text #Full description
    except AttributeError:
        pass
    if reviews:
        try:
            game_dict = {'Description': desc, 
                         'Scores': [reviews[0].text, reviews[1].text],
                         'Tags': tags, 
                         'Content': content, 
                         'Details': details, 
                         'Full description': full_desc}
        except IndexError:
            game_dict = {'Description': desc, 
                         'Scores': reviews[0].text,
                         'Tags': tags, 
                         'Content': content, 
                         'Details': details, 
                         'Full description': full_desc}
    else:
        game_dict = {'Description': desc, 
                     'Scores': '',
                     'Tags': tags, 
                     'Content': content, 
                     'Details': details, 
                     'Full description': full_desc}    
    return game_dict

In [22]:
for app in tqdm(links[6581:], total=len(links[6581:])):
    page = get_content(app)
    games_info.append(get_info(page))

<Response [502]> https://store.steampowered.com/app/763030/Nelly_Cootalot_Spoonbeaks_Ahoy_HD/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/813480/General_Horse_and_the_Package_of_Doom/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/858830/Echoes_of_the_Fey_The_Last_Sacrament/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/11250/Pacific_Storm/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/293900/Pivvot/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/994310/Lets_Sing_2019/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/423620/HERO_DEFENSE/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/1327820/The_Lost_Art_of_Innkeeping/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/386520/Champion_of_the_Gods/?snr=1_7_7_230_150_226
<Response [502]> https://store.steampowered.com/app/1093

KeyboardInterrupt: 

In [19]:
len(games_info)

6581

In [602]:
steam_apps = pd.DataFrame(games_info)

In [603]:
steam_apps['URL'] = links

In [607]:
steam_apps.to_csv('steam_apps.csv')

### Df cleaning 

In [86]:
apps = pd.read_csv('steam_apps2.csv')

In [87]:
apps.drop(['Unnamed: 0'], axis=1, inplace=True)

In [35]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53176 entries, 0 to 53175
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Description       52938 non-null  object
 1   Scores            52938 non-null  object
 2   Tags              52910 non-null  object
 3   Content           6876 non-null   object
 4   Details           53172 non-null  object
 5   Full description  52941 non-null  object
 6   URL               53176 non-null  object
dtypes: object(7)
memory usage: 2.8+ MB


In [88]:
title = re.compile('(?<=\nTitle:[ |\n]).+(?=\nGenre:)?')
genre = re.compile('(?<=\nGenre:[ |\n]).+(?=\n)')
dev = re.compile('(?<=Developer:[ |\n]).+(?=\n)')
publisher = re.compile('(?<=Publisher:[ |\n]).+(?=\n)')
release = re.compile('(?<=Release Date:[ |\n]).+(?=\n)')
franchise = re.compile('(?<=Franchise:[ |\n]).+(?=\n)')

In [7]:
steam_apps['Type'] = steam_apps['URL'].apply(lambda x: 'App' if 'app' in x else 'Bundle' if 'bundle' in x else 'Sub')

apps = steam_apps[steam_apps['Type'] == 'App']

apps.reset_index(drop=True, inplace=True)

In [89]:
apps_clean = pd.DataFrame()
apps_clean['Title'] = apps['Details'].apply(lambda x: title.search(x).group() if pd.notnull(x) and title.search(x) else '-' )

apps_clean['Genre'] = apps['Details'].apply(lambda x: genre.search(x).group() if pd.notnull(x) and genre.search(x) else '-' )

apps_clean['Developer'] = apps['Details'].apply(lambda x: dev.search(x).group() if pd.notnull(x) and dev.search(x) else '-' )

apps_clean['Publisher'] = apps['Details'].apply(lambda x: publisher.search(x).group() if pd.notnull(x) and publisher.search(x) else '-' )

apps_clean['Release'] = apps['Details'].apply(lambda x: release.search(x).group() if pd.notnull(x) and release.search(x) else '-' )

apps_clean['Franchise'] = apps['Details'].apply(lambda x: franchise.search(x).group() if pd.notnull(x) and franchise.search(x) else '-' )

In [90]:
apps_clean['Description'] = apps['Description'].apply(lambda x: re.sub('[\t\n\r]', ' ', x) if pd.notnull(x) else '-')

In [91]:
apps_clean['Full Description'] = apps['Full description'].apply(lambda x: re.sub('[\t\n\r]', ' ', x) if pd.notnull(x) else '-')
apps_clean['Full Description'] = apps_clean['Full Description'].apply(lambda x: re.sub('About This Game', ' ', x) if pd.notnull(x) else '-')

In [47]:
apps_clean.head()

,Title,Genre,Developer,Publisher,Release,Franchise,Description,Full Description
0,Counter-Strike: Global Offensive,"Action, Free to Play","Valve, Hidden Path Entertainment",Valve,"21 Aug, 2012",-,Counter-Strike: Global Offensive (CS...,Counter-Strike: Global Offensive (C...
1,Apex Legends™,"Action, Adventure, Free to Play",Respawn Entertainment,Electronic Arts,"4 Nov, 2020",Apex Legends,"Apex Legends is the award-winning, f...",Conquer with character in Apex Lege...
2,Dead by Daylight,Action,Behaviour Interactive Inc.,Behaviour Interactive Inc.,"14 Jun, 2016",-,Dead by Daylight is a multiplayer (4...,Death Is Not an Escape.Dead by Daylight is ...
3,Warframe,"Action, Free to Play",Digital Extremes,Digital Extremes,"25 Mar, 2013",-,Warframe is a cooperative free-to-pl...,CHOOSE YOUR WARFRAMEWith more than 30 Warfr...
4,Grand Theft Auto V,"Action, Adventure",Rockstar North,Rockstar Games,"14 Apr, 2015",Grand Theft Auto,Grand Theft Auto V for PC offers pla...,"When a young street hustler, a reti..."


In [92]:
apps_clean['Mature Content'] = apps['Content'].apply(
    lambda x: re.sub('[\t\n\r]', ' ', x) if pd.notnull(x) else '-')
apps_clean['Mature Content'] = apps_clean['Mature Content'].apply(
    lambda x: re.sub('Mature Content Description', ' ', x) if pd.notnull(x) else '-')
apps_clean['Mature Content'] = apps_clean['Mature Content'].apply(
    lambda x: re.sub('The developers describe the content like this:', ' ', x) if pd.notnull(x) else '-')

In [93]:
apps_clean['Mature Content'].value_counts().nlargest(5)

-                                                                                                                                                                                                         46300
          This Game may contain content not appropriate for all ages, or may not be appropriate for viewing at work: Frequent Violence or Gore, General Mature Content                                      848
          This Game may contain content not appropriate for all ages, or may not be appropriate for viewing at work: Nudity or Sexual Content, General Mature Content                                       262
          This Game may contain content not appropriate for all ages, or may not be appropriate for viewing at work: General Mature Content                                                                 212
          This Game may contain content not appropriate for all ages, or may not be appropriate for viewing at work: Nudity or Sexual Content, Frequent Violence or Gore

In [94]:
apps_clean['Tags'] = apps['Tags'].apply(
    lambda x: ', '.join(x.split('\t\t\t\t\t\t\t\t\t\t\t\t')[1::2]) if pd.notnull(x) else '-'
)

In [95]:
recent = re.compile('(?<=Recent Reviews:).+(?=All Reviews)')
all_reviews = re.compile('(?<=All Reviews:)[\n\r\t\-,\(\)\w\s%]+|(?<=All Reviews:).+')

In [96]:
apps_clean['Recent Reviews'] = apps['Scores'].apply(
    lambda x: recent.search(x).group() if pd.notnull(x) and recent.search(x) else '-'
)

In [97]:
apps_clean['All reviews'] = apps['Scores'].apply(
    lambda x: all_reviews.search(x).group() if pd.notnull(x) and all_reviews.search(x) else '-'
)

In [98]:
prct = re.compile('(?<= )\d{1,2}(?=%)')

In [99]:
apps_clean['Recent Score'] = apps_clean['Recent Reviews'].apply(lambda x: prct.search(x).group() if prct.search(x) else '-')

In [100]:
apps_clean['Scores'] = apps_clean['All reviews'].apply(lambda x: prct.search(x).group() if prct.search(x) else '-')

In [101]:
apps_clean.drop(['Recent Reviews', 'All reviews'], axis=1, inplace=True)

In [102]:
mean_score = []
for elem in list(zip(apps_clean['Recent Score'], apps_clean['Scores'])):
    if elem[0] != '-':
        mean_score.append((int(elem[0]) + int(elem[1])) // 2)
    elif elem[0] == '-' and elem[1] != '-':
        mean_score.append(int(elem[1]))
    else:
        mean_score.append('-')

In [103]:
apps_clean['Mean score'] = mean_score

In [104]:
apps_clean['URL'] = apps['URL']

In [105]:
scores_pat = re.compile(r'(?<=\().+?(?=\))')
scores_count = []
for elem in apps['Scores']:
    if type(elem) == str:
        scores = scores_pat.findall(elem)
        if scores:
            if len(scores) > 2:
                scores_count.append(scores[2])
            else:
                scores_count.append(scores[0])
        else:
            scores_count.append('-')
    else:
        scores_count.append('-')

In [106]:
apps_clean['Scores Count'] = scores_count

In [108]:
apps_clean.head(100)

,Title,Genre,Developer,Publisher,Release,Franchise,Description,Full Description,Mature Content,Tags,Recent Score,Scores,Mean score,URL,Scores Count
0,Counter-Strike: Global Offensive,"Action, Free to Play","Valve, Hidden Path Entertainment",Valve,"21 Aug, 2012",-,Counter-Strike: Global Offensive (CS...,Counter-Strike: Global Offensive (C...,Includes intense violence and blood....,"FPS, Shooter, Multiplayer, Competitive, Action...",86,88,87,https://store.steampowered.com/app/730/Counter...,"5,640,186"
1,Apex Legends™,"Action, Adventure, Free to Play",Respawn Entertainment,Electronic Arts,"4 Nov, 2020",Apex Legends,"Apex Legends is the award-winning, f...",Conquer with character in Apex Lege...,-,"Free to Play, Battle Royale, Multiplayer, Shoo...",86,90,88,https://store.steampowered.com/app/1172470/Ape...,"240,595"
2,Dead by Daylight,Action,Behaviour Interactive Inc.,Behaviour Interactive Inc.,"14 Jun, 2016",-,Dead by Daylight is a multiplayer (4...,Death Is Not an Escape.Dead by Daylight is ...,This Game may contain content not ap...,"Horror, Survival Horror, Multiplayer, Online C...",86,82,84,https://store.steampowered.com/app/381210/Dead...,"341,409"
3,Warframe,"Action, Free to Play",Digital Extremes,Digital Extremes,"25 Mar, 2013",-,Warframe is a cooperative free-to-pl...,CHOOSE YOUR WARFRAMEWith more than 30 Warfr...,This Game may contain content not ap...,"Free to Play, Looter Shooter, Action, Co-op, M...",90,91,90,https://store.steampowered.com/app/230410/Warf...,"428,036"
4,Grand Theft Auto V,"Action, Adventure",Rockstar North,Rockstar Games,"14 Apr, 2015",Grand Theft Auto,Grand Theft Auto V for PC offers pla...,"When a young street hustler, a reti...",This Game may contain content not ap...,"Open World, Action, Multiplayer, Automobile Si...",92,83,87,https://store.steampowered.com/app/271590/Gran...,"998,309"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Overcooked! 2,"Action, Casual, Indie","Ghost Town Games Ltd., Team17",Team17,"7 Aug, 2018",Team17 Digital,Overcooked returns with a brand-new ...,Overcooked returns with a brand-new...,-,"Multiplayer, Online Co-Op, Local Co-Op, Co-op,...",93,90,91,https://store.steampowered.com/app/728880/Over...,"20,983"
96,Risk of Rain 2,"Action, Indie",Hopoo Games,Gearbox Publishing,"11 Aug, 2020",-,Escape a chaotic alien planet by fig...,SURVIVE AN ALIEN PLANETOver a dozen handcra...,-,"Third-Person Shooter, Action Roguelike, Multip...",96,96,96,https://store.steampowered.com/app/632360/Risk...,"103,617"
97,Shop Titans,"Adventure, Casual, Free to Play, RPG, Simulation","Kabam Games, Inc.","Kabam Games, Inc.","5 May, 2020",-,Shop Titans is the ultimate RPG shop...,As a thriving new shop owner in a b...,-,"Free to Play, Simulation, RPG, Adventure, Casu...",82,78,80,https://store.steampowered.com/app/1258080/Sho...,"8,628"
98,Mount & Blade II: Bannerlord,"Action, RPG, Simulation, Strategy, Early Access",TaleWorlds Entertainment,TaleWorlds Entertainment,"30 Mar, 2020",-,"The horns sound, the ravens gather. ...","The horns sound, the ravens gather. An empi...",This game is set in a fictional medi...,"Medieval, Strategy, Open World, RPG, War, Mult...",81,87,84,https://store.steampowered.com/app/261550/Moun...,"127,569"


In [109]:
apps_clean = apps_clean[apps_clean['Title'] != '-']

In [120]:
apps_clean['isFree'] = apps_clean['Genre'].apply(lambda x: 'Free' if re.search('Free', x) else 'payed') 

In [130]:
apps_clean.to_csv('apps2.csv')